In [ ]:
pip install CityLearn

In [ ]:
pip install stable_baselines3

In [2]:
import citylearn

In [3]:
citylearn.__version__

'2.3.0'

In [7]:
pip install numpy --upgrade

In [4]:
# System operations
import inspect
import os
import uuid

# Date and time
from datetime import datetime

# type hinting
from typing import List, Mapping, Tuple

# Data visualization
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from tqdm import tqdm

# data manipulation
from bs4 import BeautifulSoup
import math
import numpy as np
import pandas as pd
import random
import re
import requests
import simplejson as json

# cityLearn
from citylearn.agents.base import (
   BaselineAgent,
   Agent as RandomAgent
)
from citylearn.agents.rbc import HourRBC
from citylearn.agents.q_learning import TabularQLearning
from citylearn.citylearn import CityLearnEnv
from citylearn.data import DataSet
from citylearn.reward_function import RewardFunction
from citylearn.wrappers import (
    NormalizedObservationWrapper,
    StableBaselines3Wrapper,
    TabularQLearningWrapper,
    NormalizedActionWrapper,
    StableBaselines3ObservationWrapper
)

# RL algorithms
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import BaseCallback

# User interaction
from IPython.display import clear_output
from ipywidgets import Button, FloatSlider, HBox, HTML
from ipywidgets import IntProgress, Text, VBox

Couldn't import dot_parser, loading of dot files will not be possible.


In [5]:
print('All CityLearn datasets:', sorted(DataSet().get_dataset_names()))

INFO:root:The dataset names DNE in cache. Will download from intelligent-environments-lab/CityLearn/tree/v2.3.0 GitHub repository and write to /home/RDC/ivanovsi/.cache/citylearn/v2.3.0/dataset_names.json. Next time DataSet.get_dataset_names is called, it will read from cache unless DataSet.clear_cache is run first.


All CityLearn datasets: ['baeda_3dem', 'ca_alameda_county_neighborhood', 'citylearn_challenge_2020_climate_zone_1', 'citylearn_challenge_2020_climate_zone_2', 'citylearn_challenge_2020_climate_zone_3', 'citylearn_challenge_2020_climate_zone_4', 'citylearn_challenge_2021', 'citylearn_challenge_2022_phase_1', 'citylearn_challenge_2022_phase_2', 'citylearn_challenge_2022_phase_3', 'citylearn_challenge_2022_phase_all', 'citylearn_challenge_2022_phase_all_plus_evs', 'citylearn_challenge_2023_phase_1', 'citylearn_challenge_2023_phase_2_local_evaluation', 'citylearn_challenge_2023_phase_2_online_evaluation_1', 'citylearn_challenge_2023_phase_2_online_evaluation_2', 'citylearn_challenge_2023_phase_2_online_evaluation_3', 'citylearn_challenge_2023_phase_3_1', 'citylearn_challenge_2023_phase_3_2', 'citylearn_challenge_2023_phase_3_3', 'quebec_neighborhood_with_demand_response_set_points', 'quebec_neighborhood_without_demand_response_set_points', 'tx_travis_county_neighborhood', 'vt_chittenden_co

In [6]:
dataset_path = "citylearn_challenge_2023_phase_1"  # Path to the dataset directory

# Initialize the CityLearnEnv with the path
env = CityLearnEnv(dataset_path)

INFO:root:The citylearn_challenge_2023_phase_1 dataset DNE in cache. Will download from intelligent-environments-lab/CityLearn/tree/v2.3.0 GitHub repository and write to /home/RDC/ivanovsi/.cache/citylearn/v2.3.0/datasets. Next time DataSet.get_dataset('citylearn_challenge_2023_phase_1') is called, it will read from cache unless DataSet.clear_cache is run first.
INFO:root:The PV sizing data DNE in cache. Will download from intelligent-environments-lab/CityLearn/tree/v2.3.0 GitHub repository and write to /home/RDC/ivanovsi/.cache/citylearn/v2.3.0/misc. Next time DataSet.get_pv_sizing_data is called, it will read from cache unless DataSet.clear_cache is run first.
INFO:root:The battery sizing data DNE in cache. Will download from intelligent-environments-lab/CityLearn/tree/v2.3.0 GitHub repository and write to /home/RDC/ivanovsi/.cache/citylearn/v2.3.0/misc. Next time DataSet.get_battery_sizing_data is called, it will read from cache unless DataSet.clear_cache is run first.


In [7]:
# Check number of buildings in the environment
print(f"Number of buildings: {len(env.buildings)}")

# Check the number of time steps or a sample of observations
for i, b in enumerate(env.buildings):
    print(f"Observations for Building {i}: {b.observations()}")

Number of buildings: 3
Observations for Building 0: {'day_type': 5, 'hour': 1, 'outdoor_dry_bulb_temperature': 24.66, 'outdoor_dry_bulb_temperature_predicted_1': 24.910639, 'outdoor_dry_bulb_temperature_predicted_2': 38.41596, 'outdoor_dry_bulb_temperature_predicted_3': 27.611464, 'diffuse_solar_irradiance': 0.0, 'diffuse_solar_irradiance_predicted_1': 54.625927, 'diffuse_solar_irradiance_predicted_2': 116.84289, 'diffuse_solar_irradiance_predicted_3': 0.0, 'direct_solar_irradiance': 0.0, 'direct_solar_irradiance_predicted_1': 143.32434, 'direct_solar_irradiance_predicted_2': 1020.7561, 'direct_solar_irradiance_predicted_3': 0.0, 'carbon_intensity': 0.40248835, 'indoor_dry_bulb_temperature': 23.098652, 'non_shiftable_load': 0.35683933, 'solar_generation': 0.0, 'dhw_storage_soc': 0.0, 'electrical_storage_soc': 0.2, 'net_electricity_consumption': 0.67788136, 'electricity_pricing': 0.02893, 'electricity_pricing_predicted_1': 0.02893, 'electricity_pricing_predicted_2': 0.02915, 'electricit

In [8]:
data = []
for i, b in enumerate(env.buildings):
    for t in range(env.time_steps):  # Iterate over the time steps (adjust this depending on the time steps available)
        obs = b.observations(t)  # Assuming you can specify a time step
        data.append({"Building": i, "Time Step": t, "Observations": obs})

# Convert to DataFrame
df = pd.DataFrame(data)
print(f"Total rows: {len(df)}")  # Check if you have more than 3 rows now
print(df.head())


Total rows: 2160
   Building  Time Step                                       Observations
0         0          0  {'day_type': 5, 'hour': 1, 'outdoor_dry_bulb_t...
1         0          1  {'month': 6, 'hour': 1, 'day_type': 5, 'indoor...
2         0          2  {'month': 6, 'hour': 1, 'day_type': 5, 'indoor...
3         0          3  {'month': 6, 'hour': 1, 'day_type': 5, 'indoor...
4         0          4  {'month': 6, 'hour': 1, 'day_type': 5, 'indoor...


In [ ]:
citylearn.wrappers.NormalizedObservationWrapper(env)

<NormalizedObservationWrapper<CityLearnEnv instance>>

In [ ]:
citylearn.wrappers.ClippedObservationWrapper(env)

<ClippedObservationWrapper<CityLearnEnv instance>>

In [18]:
from stable_baselines3.sac import SAC as Agent
from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

# initialize
env = CityLearnEnv('citylearn_challenge_2023_phase_2_local_evaluation', central_agent=True)
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
model = Agent('MlpPolicy', env)

# train
episodes = 5
model.learn(total_timesteps=env.unwrapped.time_steps*episodes)

# test
observations, _ = env.reset()

while not env.unwrapped.terminated:
    actions, _ = model.predict(observations, deterministic=True)
    observations, _, _, _, _ = env.step(actions)

kpis = env.unwrapped.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value').round(3)
kpis = kpis.dropna(how='all')
display(kpis)

INFO:root:The citylearn_challenge_2023_phase_2_local_evaluation dataset DNE in cache. Will download from intelligent-environments-lab/CityLearn/tree/v2.3.0 GitHub repository and write to /home/RDC/ivanovsi/.cache/citylearn/v2.3.0/datasets. Next time DataSet.get_dataset('citylearn_challenge_2023_phase_2_local_evaluation') is called, it will read from cache unless DataSet.clear_cache is run first.


name                                           Building_1  Building_2  \
cost_function                                                           
all_time_peak_average                                 NaN         NaN   
annual_normalized_unserved_energy_total             0.014       0.008   
carbon_emissions_total                              0.414       0.733   
cost_total                                          0.392       0.759   
daily_one_minus_load_factor_average                   NaN         NaN   
daily_peak_average                                    NaN         NaN   
discomfort_cold_delta_average                       0.000       0.004   
discomfort_cold_delta_maximum                       0.124       0.581   
discomfort_cold_delta_minimum                       0.000       0.000   
discomfort_cold_proportion                          0.000       0.000   
discomfort_hot_delta_average                        9.461       2.803   
discomfort_hot_delta_maximum                       16.711      10.743   
discomfort_hot_delta_minimum                        0.000       0.000   
discomfort_hot_proportion                           0.982       0.841   
discomfort_proportion                               0.982       0.841   
electricity_consumption_total                       0.418       0.734   
monthly_one_minus_load_factor_average                 NaN         NaN   
one_minus_thermal_resilience_proportion             1.000       1.000   
power_outage_normalized_unserved_energy_total       0.528       0.502   
ramping_average                                       NaN         NaN   
zero_net_energy                                     0.273       0.734   

name                                           Building_3  District  
cost_function                                                        
all_time_peak_average                                 NaN     0.970  
annual_normalized_unserved_energy_total             0.016     0.012  
carbon_emissions_total                              1.610     0.919  
cost_total                                          1.615     0.922  
daily_one_minus_load_factor_average                   NaN     1.047  
daily_peak_average                                    NaN     0.929  
discomfort_cold_delta_average                       2.970     0.991  
discomfort_cold_delta_maximum                       6.825     2.510  
discomfort_cold_delta_minimum                       0.000     0.000  
discomfort_cold_proportion                          0.834     0.278  
discomfort_hot_delta_average                        0.012     4.092  
discomfort_hot_delta_maximum                        3.560    10.338  
discomfort_hot_delta_minimum                        0.000     0.000  
discomfort_hot_proportion                           0.003     0.609  
discomfort_proportion                               0.837     0.887  
electricity_consumption_total                       1.615     0.922  
monthly_one_minus_load_factor_average                 NaN     1.023  
one_minus_thermal_resilience_proportion             0.133     0.711  
power_outage_normalized_unserved_energy_total       0.732     0.587  
ramping_average                                       NaN     0.940  
zero_net_energy                                     1.631     0.879

In [ ]:
import shimmy

In [ ]:
pip install 'shimmy>=2.0'

In [ ]:
from citylearn.agents.base import BaselineAgent as Agent
from citylearn.citylearn import CityLearnEnv

# initialize
env = CityLearnEnv('citylearn_challenge_2023_phase_2_local_evaluation', central_agent=True)
model = Agent(env)

# step through environment and apply agent actions
observations, _ = env.reset()

while not env.terminated:
    actions = model.predict(observations)
    observations, reward, info, terminated, truncated = env.step(actions)

# test
kpis = model.env.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value').round(3)
kpis = kpis.dropna(how='all')
display(kpis)

INFO:root:The citylearn_challenge_2023_phase_2_local_evaluation dataset DNE in cache. Will download from intelligent-environments-lab/CityLearn/tree/v2.3.0 GitHub repository and write to /root/.cache/citylearn/v2.3.0/datasets. Next time DataSet.get_dataset('citylearn_challenge_2023_phase_2_local_evaluation') is called, it will read from cache unless DataSet.clear_cache is run first.
/usr/local/lib/python3.11/dist-packages/citylearn/dynamics.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to b

name                                           Building_1  Building_2  \
cost_function                                                           
all_time_peak_average                                 NaN         NaN   
annual_normalized_unserved_energy_total             0.019       0.018   
carbon_emissions_total                              1.000       1.000   
cost_total                                          1.000       1.000   
daily_one_minus_load_factor_average                   NaN         NaN   
daily_peak_average                                    NaN         NaN   
discomfort_cold_delta_average                       1.657       0.045   
discomfort_cold_delta_maximum                       4.878       1.793   
discomfort_cold_delta_minimum                       0.000       0.000   
discomfort_cold_proportion                          0.369       0.000   
discomfort_hot_delta_average                        0.053       0.577   
discomfort_hot_delta_maximum                        5.006       6.441   
discomfort_hot_delta_minimum                        0.000       0.000   
discomfort_hot_proportion                           0.008       0.041   
discomfort_proportion                               0.377       0.041   
electricity_consumption_total                       1.000       1.000   
monthly_one_minus_load_factor_average                 NaN         NaN   
one_minus_thermal_resilience_proportion             0.333       0.643   
power_outage_normalized_unserved_energy_total       0.723       0.692   
ramping_average                                       NaN         NaN   
zero_net_energy                                     1.000       1.000   

name                                           Building_3  District  
cost_function                                                        
all_time_peak_average                                 NaN     1.000  
annual_normalized_unserved_energy_total             0.018     0.018  
carbon_emissions_total                              1.000     1.000  
cost_total                                          1.000     1.000  
daily_one_minus_load_factor_average                   NaN     1.000  
daily_peak_average                                    NaN     1.000  
discomfort_cold_delta_average                       0.675     0.793  
discomfort_cold_delta_maximum                       3.642     3.438  
discomfort_cold_delta_minimum                       0.000     0.000  
discomfort_cold_proportion                          0.095     0.155  
discomfort_hot_delta_average                        0.081     0.237  
discomfort_hot_delta_maximum                        3.734     5.060  
discomfort_hot_delta_minimum                        0.000     0.000  
discomfort_hot_proportion                           0.003     0.018  
discomfort_proportion                               0.098     0.172  
electricity_consumption_total                       1.000     1.000  
monthly_one_minus_load_factor_average                 NaN     1.000  
one_minus_thermal_resilience_proportion             0.133     0.370  
power_outage_normalized_unserved_energy_total       0.637     0.684  
ramping_average                                       NaN     1.000  
zero_net_energy                                     1.000     1.000

In [20]:
pip install gym
import gym
import citylearn
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback

SyntaxError: invalid syntax (1331984234.py, line 1)

In [ ]:
env = NormalizedObservationWrapper(env)
env = NormalizedActionWrapper(env)

env1 = StableBaselines3ObservationWrapper(env)
env = DummyVecEnv([lambda: env1])

/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [ ]:
print('All CityLearn datasets:', sorted(DataSet().get_dataset_names()))

All CityLearn datasets: ['baeda_3dem', 'ca_alameda_county_neighborhood', 'citylearn_challenge_2020_climate_zone_1', 'citylearn_challenge_2020_climate_zone_2', 'citylearn_challenge_2020_climate_zone_3', 'citylearn_challenge_2020_climate_zone_4', 'citylearn_challenge_2021', 'citylearn_challenge_2022_phase_1', 'citylearn_challenge_2022_phase_2', 'citylearn_challenge_2022_phase_3', 'citylearn_challenge_2022_phase_all', 'citylearn_challenge_2022_phase_all_plus_evs', 'citylearn_challenge_2023_phase_1', 'citylearn_challenge_2023_phase_2_local_evaluation', 'citylearn_challenge_2023_phase_2_online_evaluation_1', 'citylearn_challenge_2023_phase_2_online_evaluation_2', 'citylearn_challenge_2023_phase_2_online_evaluation_3', 'citylearn_challenge_2023_phase_3_1', 'citylearn_challenge_2023_phase_3_2', 'citylearn_challenge_2023_phase_3_3', 'quebec_neighborhood_with_demand_response_set_points', 'quebec_neighborhood_without_demand_response_set_points', 'tx_travis_county_neighborhood', 'vt_chittenden_co

In [ ]:
dataset_path = "citylearn_challenge_2023_phase_1"  # Path to the dataset directory

# Initialize the CityLearnEnv with the path
env = CityLearnEnv(dataset_path)

In [31]:
from citylearn.agents.base import BaselineAgent as Agent
from citylearn.citylearn import CityLearnEnv

# initialize
env = CityLearnEnv('citylearn_challenge_2023_phase_2_online_evaluation_1', central_agent=True)
model = Agent(env)

# step through environment and apply agent actions
observations, _ = env.reset()

while not env.terminated:
    actions = model.predict(observations)
    observations, reward, info, terminated, truncated = env.step(actions)

# test
kpis = model.env.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value').round(3)
kpis = kpis.dropna(how='all')
display(kpis)

/home/RDC/ivanovsi/.local/lib/python3.11/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/RDC/ivanovsi/.local/lib/python3.11/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


name                                           Building_1  Building_2  \
cost_function                                                           
all_time_peak_average                                 NaN         NaN   
annual_normalized_unserved_energy_total             0.006       0.006   
carbon_emissions_total                              1.000       1.000   
cost_total                                          1.000       1.000   
daily_one_minus_load_factor_average                   NaN         NaN   
daily_peak_average                                    NaN         NaN   
discomfort_cold_delta_average                       1.450       0.042   
discomfort_cold_delta_maximum                       4.771       2.826   
discomfort_cold_delta_minimum                       0.000       0.000   
discomfort_cold_proportion                          0.294       0.004   
discomfort_hot_delta_average                        0.117       0.493   
discomfort_hot_delta_maximum                        6.580       6.192   
discomfort_hot_delta_minimum                        0.000       0.000   
discomfort_hot_proportion                           0.017       0.030   
discomfort_proportion                               0.311       0.034   
electricity_consumption_total                       1.000       1.000   
monthly_one_minus_load_factor_average                 NaN         NaN   
one_minus_thermal_resilience_proportion             0.667       1.000   
power_outage_normalized_unserved_energy_total       0.709       0.698   
ramping_average                                       NaN         NaN   
zero_net_energy                                     1.000       1.000   

name                                           Building_3  District  
cost_function                                                        
all_time_peak_average                                 NaN     1.000  
annual_normalized_unserved_energy_total             0.005     0.006  
carbon_emissions_total                              1.000     1.000  
cost_total                                          1.000     1.000  
daily_one_minus_load_factor_average                   NaN     1.000  
daily_peak_average                                    NaN     1.000  
discomfort_cold_delta_average                       0.366     0.619  
discomfort_cold_delta_maximum                       3.112     3.570  
discomfort_cold_delta_minimum                       0.000     0.000  
discomfort_cold_proportion                          0.020     0.106  
discomfort_hot_delta_average                        0.286     0.298  
discomfort_hot_delta_maximum                        3.518     5.430  
discomfort_hot_delta_minimum                        0.000     0.000  
discomfort_hot_proportion                           0.034     0.027  
discomfort_proportion                               0.054     0.133  
electricity_consumption_total                       1.000     1.000  
monthly_one_minus_load_factor_average                 NaN     1.000  
one_minus_thermal_resilience_proportion             0.750     0.806  
power_outage_normalized_unserved_energy_total       0.632     0.680  
ramping_average                                       NaN     1.000  
zero_net_energy                                     1.000     1.000

In [22]:
from citylearn.agents.sac import SAC as Agent
from citylearn.citylearn import CityLearnEnv

# Define which buildings to train on (inside phase 1 dataset)
building_scenarios = [
    [0],  # Train on Building 0
    [1],  # Train on Building 1
    [2]   # Train on Building 2
]

models2 = []

for building_set in building_scenarios:
    # Initialize CityLearn Environment with specific buildings
    env = CityLearnEnv(schema='citylearn_challenge_2023_phase_2_online_evaluation_1',
                        building_ids=building_set, central_agent=False)

    # Get observation shape (for first building)
    obs_shape = env.observation_space[0].shape[0]  # Correct way to access observation shape
    print(f"Training on Buildings: {building_set}, Observation Size: {obs_shape}")

    # Train SAC Agent
    model2 = Agent(env)
    model2.learn(episodes=5, deterministic_finish=True)

    # Store trained models for later adaptation
    models2.append(model2)

print("✅ Meta-RL training completed across multiple buildings!")


INFO:root:The citylearn_challenge_2023_phase_2_online_evaluation_1 dataset DNE in cache. Will download from intelligent-environments-lab/CityLearn/tree/v2.3.0 GitHub repository and write to /home/RDC/ivanovsi/.cache/citylearn/v2.3.0/datasets. Next time DataSet.get_dataset('citylearn_challenge_2023_phase_2_online_evaluation_1') is called, it will read from cache unless DataSet.clear_cache is run first.


Training on Buildings: [0], Observation Size: 30


/home/RDC/ivanovsi/.local/lib/python3.11/site-packages/citylearn/agents/sac.py:117: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:78.)
  o = tensor(o).to(self.device)


Training on Buildings: [1], Observation Size: 30
Training on Buildings: [2], Observation Size: 30
✅ Meta-RL training completed across multiple buildings!


In [32]:
# Load an unseen building (Building 3)
test_env = CityLearnEnv(schema='citylearn_challenge_2023_phase_2_online_evaluation_1', building_ids=[2], central_agent=False)

# Use one of the trained models (example: model trained on Building 0)
meta_model = models2[0]

# Reset environment
obs, _ = test_env.reset()
done = False

while not done:
    actions  = meta_model.predict(obs, deterministic=True)
    obs, _, done, _, _ = test_env.step(actions)

# Evaluate KPIs for the unseen building
kpis = test_env.evaluate()
kpis = kpis.pivot(index="cost_function", columns="name", values="value").round(3)
kpis = kpis.dropna(how="all")

# Display KPIs
print("KPIs for Unseen Building:")
display(kpis)

KPIs for Unseen Building:


name                                           Building_1  Building_2  \
cost_function                                                           
all_time_peak_average                                 NaN         NaN   
annual_normalized_unserved_energy_total             0.008       0.005   
carbon_emissions_total                              0.805       1.070   
cost_total                                          0.777       1.052   
daily_one_minus_load_factor_average                   NaN         NaN   
daily_peak_average                                    NaN         NaN   
discomfort_cold_delta_average                       0.059       0.049   
discomfort_cold_delta_maximum                       2.519       2.826   
discomfort_cold_delta_minimum                       0.000       0.000   
discomfort_cold_proportion                          0.002       0.003   
discomfort_hot_delta_average                        1.042       0.342   
discomfort_hot_delta_maximum                        8.306       6.518   
discomfort_hot_delta_minimum                        0.000       0.000   
discomfort_hot_proportion                           0.133       0.023   
discomfort_proportion                               0.135       0.027   
electricity_consumption_total                       0.807       1.071   
monthly_one_minus_load_factor_average                 NaN         NaN   
one_minus_thermal_resilience_proportion             0.867       1.000   
power_outage_normalized_unserved_energy_total       0.726       0.659   
ramping_average                                       NaN         NaN   
zero_net_energy                                     0.780       1.043   

name                                           Building_3  District  
cost_function                                                        
all_time_peak_average                                 NaN     0.907  
annual_normalized_unserved_energy_total             0.007     0.007  
carbon_emissions_total                              0.946     0.940  
cost_total                                          0.925     0.918  
daily_one_minus_load_factor_average                   NaN     1.002  
daily_peak_average                                    NaN     0.920  
discomfort_cold_delta_average                       0.039     0.049  
discomfort_cold_delta_maximum                       1.639     2.328  
discomfort_cold_delta_minimum                       0.000     0.000  
discomfort_cold_proportion                          0.000     0.002  
discomfort_hot_delta_average                        0.309     0.565  
discomfort_hot_delta_maximum                        3.725     6.183  
discomfort_hot_delta_minimum                        0.000     0.000  
discomfort_hot_proportion                           0.007     0.054  
discomfort_proportion                               0.007     0.056  
electricity_consumption_total                       0.945     0.941  
monthly_one_minus_load_factor_average                 NaN     1.061  
one_minus_thermal_resilience_proportion             0.750     0.872  
power_outage_normalized_unserved_energy_total       0.662     0.682  
ramping_average                                       NaN     1.027  
zero_net_energy                                     0.944     0.922